# Beamline alignment in Bluesky
This notebook will cover the commands needed to run alignment scans in Bluesky.  

Two cases will be covered: 

1: Aligning using a point detector

2: Aligning using an area detector

In [2]:
# Initialize the environment
from ssrlsim.scripts.start_RE import *
#%matplotlib widget

# Expose motors and detectors for this simulation
from ssrlsim.hitp_waxs import *

# We've gathered most of the tools you might need to run your beamline under the ssrltools package.  
# For alignment try the this stage leveling plan
from ssrltools.plans import level_stage_single

Filestore path: C:\Users\roberttk\Desktop\SLAC_RA\bluesky-dev\ssrlsim\notebooks\fstore


In [26]:
plt.ion()

# Device summary
The motors for the high throughput stage fixture are organized in an object named `stage`.  Individual motors can be accessed via `stage.stage_x`, `stage.plate_x` etc.  The laser range finder is named `lrf`, and will react to movements made by the HiTp stage motors.  The beamstop detector is named `ptDet` and can be occluded by the HiTp stage height: `stage.stage_z`

In [2]:
# play with various stage and detector objects here
stage.component_names

('stage_x', 'stage_y', 'stage_z', 'plate_x', 'plate_y', 'theta')

In [3]:
# Can read any of these with the .read() method
lrf.read()

{'lrf': {'value': 2.980371, 'timestamp': 1591647785.2834013}}

In [4]:
# Each motor/device has a name that Bluesky identifies it by
ptDet.name 

'ptDet'

# Level stage using pico motors and laser range finder
We have built a plan to automatically level the stage along one direction

In [27]:
# Move to center of stage first
RE(bps.mv(stage.stage_x, 0, stage.stage_y, 0)) # Can also split into two different commands 

# Look at profile of stage before alignment
RE(bp.scan([lrf], stage.stage_x, -30, 30, num=20))

Transient Scan ID: 6     Time: 2020-06-08 14:00:22
Persistent Unique Scan ID: '916f4194-db27-4e30-ad9c-05621b40c647'
New stream: 'primary'
+-----------+------------+--------------------+------------+
|   seq_num |       time | HiTp_Stage_stage_x |        lrf |
+-----------+------------+--------------------+------------+
|         1 | 14:00:22.0 |            -30.000 |      3.917 |
|         2 | 14:00:22.1 |            -26.842 |      3.919 |
|         3 | 14:00:22.1 |            -23.684 |      3.922 |
|         4 | 14:00:22.1 |            -20.526 |      3.924 |
|         5 | 14:00:22.1 |            -17.368 |      3.926 |
|         6 | 14:00:22.1 |            -14.211 |      3.928 |
|         7 | 14:00:22.2 |            -11.053 |      3.930 |
|         8 | 14:00:22.2 |             -7.895 |      3.933 |
|         9 | 14:00:22.2 |             -4.737 |      2.935 |
|        10 | 14:00:22.2 |             -1.579 |      2.937 |
|        11 | 14:00:22.2 |              1.579 |      2.939 |
|      

('916f4194-db27-4e30-ad9c-05621b40c647',)

In [19]:
# move to center of stage 
RE(bps.mv(stage.stage_x, 0, stage.stage_y, 0))
# Level x-axis
RE(level_stage_single(lrf, stage.plate_x, stage.stage_x, -30, 30))

0 iters for thresh_mult=20
0 iters for thresh_mult=10
5 iters for thresh_mult=5
17 iters for thresh_mult=2


()

In [23]:
# Re-examine stage to verify it has been leveled.  Should be level within a tolerance of 0.0427V (29.4 steps)
RE(bp.scan([lrf], stage.stage_x, -30, 30, num=20))

Transient Scan ID: 4     Time: 2020-06-08 13:37:53
Persistent Unique Scan ID: 'c07b21e2-1bb4-4436-a295-c28272ad2759'
New stream: 'primary'
+-----------+------------+--------------------+------------+
|   seq_num |       time | HiTp_Stage_stage_x |        lrf |
+-----------+------------+--------------------+------------+
|         1 | 13:37:53.9 |            -30.000 |      3.917 |
|         2 | 13:37:53.9 |            -26.842 |      3.919 |
|         3 | 13:37:53.9 |            -23.684 |      3.922 |
|         4 | 13:37:53.9 |            -20.526 |      3.924 |
|         5 | 13:37:54.0 |            -17.368 |      3.926 |
|         6 | 13:37:54.0 |            -14.211 |      3.928 |
|         7 | 13:37:54.0 |            -11.053 |      3.930 |
|         8 | 13:37:54.0 |             -7.895 |      3.933 |
|         9 | 13:37:54.0 |             -4.737 |      2.935 |
|        10 | 13:37:54.0 |             -1.579 |      2.937 |
|        11 | 13:37:54.0 |              1.579 |      2.939 |
|      

('c07b21e2-1bb4-4436-a295-c28272ad2759',)

In [22]:
RE(bps.mv(stage.stage_x, -30))
v1 = lrf.read()['lrf']['value']
RE(bps.mv(stage.stage_x, 30))
v2 = lrf.read()['lrf']['value']
print(v1-v2)

-0.04180250000000019


In [15]:
RE(bps.mv(stage.stage_x, 30))
lrf.read()

{'lrf': {'value': 3.9821885, 'timestamp': 1591381340.9422255}}

In [ ]:
# As an exercise, try writing the commands necessary to level the y-stage


# Align Z-stage using ims stage motors and beamstop detector 
This alignment procedure is significantly less complicated, so we'll do this one manually

In [24]:
# Scan the z-stage motor while watching the point detector
RE( bp.scan([ptDet], stage.stage_z, -5, 5, num=40) )

Transient Scan ID: 5     Time: 2020-06-08 13:58:17
Persistent Unique Scan ID: 'a09d479c-cdae-4829-b718-0fe33e28446f'
New stream: 'primary'


FigureCanvasNbAgg()

+-----------+------------+--------------------+------------+
|   seq_num |       time | HiTp_Stage_stage_z |      ptDet |
+-----------+------------+--------------------+------------+
|         1 | 13:58:17.6 |             -5.000 |      1.000 |
|         2 | 13:58:17.7 |             -4.744 |      1.000 |
|         3 | 13:58:17.7 |             -4.487 |      1.000 |
|         4 | 13:58:17.7 |             -4.231 |      1.000 |
|         5 | 13:58:17.7 |             -3.974 |      1.000 |
|         6 | 13:58:17.7 |             -3.718 |      1.000 |
|         7 | 13:58:17.7 |             -3.462 |      1.000 |
|         8 | 13:58:17.7 |             -3.205 |      1.000 |
|         9 | 13:58:17.8 |             -2.949 |      1.000 |
|        10 | 13:58:17.8 |             -2.692 |      1.000 |
|        11 | 13:58:17.8 |             -2.436 |      1.000 |
|        12 | 13:58:17.8 |             -2.179 |      1.000 |
|        13 | 13:58:17.8 |             -1.923 |      1.000 |
|        14 | 13:58:17.8

('a09d479c-cdae-4829-b718-0fe33e28446f',)

In [4]:
# peak scan information is automatically saved under the variable "peaks" after every run
peaks

{
'com':
    {'ptDet': 5.0}
,
'cen':
    {'ptDet': 2.9932061366715894}
,
'max':
    {'ptDet': (-5.0,
               0.9999999998076405)}
,
'min':
    {'ptDet': (5.0,
               -3.987867874821374)}
,
'fwhm':
    {'ptDet': None}
,
'nlls':
    {'ptDet': None}
,
}